In [ ]:
# Load the saved datasets
train_data = np.load('./data/splits/train_data.npy')
val_data = np.load('./data/splits/val_data.npy')
test_data = np.load('./data/splits/test_data.npy')

train_targets = np.load('./data/splits/train_targets.npy')
val_targets = np.load('./data/splits/val_targets.npy')
test_targets = np.load('./data/splits/test_targets.npy')